In [68]:
import os
import numpy as np
import pandas as pd
import torch
from torch import nn


In [69]:
data=pd.read_csv("data/ner_dataset.csv",encoding = "ISO-8859-1")
data.shape

(1048575, 4)

In [70]:
def load_data(file_path):
    with open(file_path,"r") as file:
        data=[ line.strip() for line in file.readlines()] 
    return data
train_sentence=load_data('data/large/train/sentences.txt')
train_labels=load_data('data/large/train/labels.txt')

val_sentence=load_data('data/large/val/sentences.txt')
val_labels=load_data('data/large/val/labels.txt')
test_sentence=load_data('data/large/test/sentences.txt')
test_labels=load_data('data/large/test/labels.txt')
    

In [71]:
def create_vocab(sentences):
    vocab={}
    vocab["<unk>"]=0
    i=1
    for sentence in sentences:
        sentence=sentence.strip().split()
        for word in sentence:
            if word not in vocab.keys():
                vocab[word]=i
                i+=1
    return vocab

vocab=create_vocab(train_sentence)
label_vocab=create_vocab(train_sentence)

In [72]:
def vectorized_sentence(vocab,sentences):
    vectorized_sentences=[]
    max_len=0
    for i,sentence in enumerate(sentences):
        sentence=sentence.strip().split()
        max_len=np.maximum(max_len,len(sentence))
        vectorized=[vocab.get(word,0) for word in sentence]
        vectorized_sentences.append(torch.tensor(vectorized))
    return vectorized_sentences,max_len

In [73]:
train_data,max_len1=vectorized_sentence(vocab,train_sentence)
test_data,max_len2=vectorized_sentence(vocab,test_sentence)
train_label_data,_=vectorized_sentence(label_vocab,train_labels)
test_label_data,_=vectorized_sentence(label_vocab,test_labels)

In [74]:
max_len=0
max_len=np.maximum(max_len,max_len1)
max_len=np.maximum(max_len,max_len2)

In [75]:
from torch.nn.utils.rnn import pad_sequence
len_train=len(train_data)
data_for_padding=train_data+test_data
data=pad_sequence(data_for_padding,padding_value=0.0,batch_first=True)
X_train=data[:len_train]
X_test=data[len_train:]


In [76]:
data_for_padding2=train_label_data+test_label_data
data1=pad_sequence(data_for_padding2,padding_value=0,batch_first=True)
y_train=data1[:len_train]
y_test=data1[len_train:]


In [77]:
train_dataset=torch.utils.data.TensorDataset(X_train,y_train)
test_dataset=torch.utils.data.TensorDataset(X_test,y_test)


In [78]:
train_loader=torch.utils.data.DataLoader(train_dataset,batch_size=64,shuffle=True)
test_loader=torch.utils.data.DataLoader(test_dataset,batch_size=64,shuffle=True)


In [108]:
embedding_size=64
hidden_size=128


class Ner_model(nn.Module):
    def __init__(self,vocab_size,embedding_size,output_size,hidden_size):
        super(Ner_model,self).__init__()
        self.embedding=nn.Embedding(vocab_size,embedding_size,padding_idx=0)
        self.rnn=nn.LSTM(embedding_size,hidden_size,num_layers=2,batch_first=True)
        self.out=nn.Linear(hidden_size,output_size)
    def forward(self,x):
        x=self.embedding(x)
        x,_=self.rnn(x)
        x=self.out(x)
        return x.permute(0,2,1)
model=Ner_model(len(vocab),embedding_size,output_size=len(label_vocab),hidden_size=hidden_size)

In [109]:
loss=torch.nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.001)

In [112]:
from engine import train
device="cuda" if torch.cuda.is_available() else 'cpu'
device


'cpu'

In [107]:
result=train(model,train_loader,test_loader,optimizer,loss,5,device)

torch.Size([64, 104, 29846])
torch.Size([64, 104])


tensor(10.2502, grad_fn=<NllLoss2DBackward0>)

torch.int64

In [1]:
import torch


In [2]:
torch.cuda.is_available()

False